In [1]:
# 导入相关数据包
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

data = pd.read_csv('./data/output/data_transform_outV2/part-00000', sep=',', 
                   names = ["device_id", "vc_id", "vd_id", 
                            "net_droppedTx_summation", 
                            "net_packetsTx_summation", 
                            "virtualDisk_numberReadAveraged_average", 
                            "virtualDisk_numberWriteAveraged_average",
                            "virtualDisk_write_average",
                            "virtualDisk_read_average",
                            "datastore_maxTotalLatency_latest",
                            "cpu_usage_average",
                            "mem_usage_average",
                            "net_usage_average",
                            "net_packetsRx_summation",
                            "net_droppedRx_summation",
                            "net_bytesRx_average",
                            "net_bytesTx_average",
                            "time",
                            "date"
                           ])

for i in range(1, 7):
    file_path = './data/output/data_transform_outV2/part-0000{}'.format(i)
    d = pd.read_csv(file_path, sep=',', 
                   names = ["device_id", "vc_id", "vd_id", 
                            "net_droppedTx_summation", 
                            "net_packetsTx_summation", 
                            "virtualDisk_numberReadAveraged_average", 
                            "virtualDisk_numberWriteAveraged_average",
                            "virtualDisk_write_average",
                            "virtualDisk_read_average",
                            "datastore_maxTotalLatency_latest",
                            "cpu_usage_average",
                            "mem_usage_average",
                            "net_usage_average",
                            "net_packetsRx_summation",
                            "net_droppedRx_summation",
                            "net_bytesRx_average",
                            "net_bytesTx_average",
                            "time",
                            "date"
                           ])
    data = data.append(d)

In [2]:
device_2400 = data.loc[data['device_id'] == 2400]
device_2400 = device_2400.reset_index().drop('index', axis=1)
device_2400 = device_2400.drop('device_id', axis=1).drop('vc_id', axis=1).drop('vd_id', axis=1).drop('time', axis=1)
device_2400_day = device_2400.groupby(['date']).sum()
index = device_2400_day.index
device_2400_day.index = np.arange(device_2400_day.shape[0])

In [3]:
def date2num(dates):
    indexs = []
    for d in dates:
        month = int(d.split('-')[1])
        day = int(d.split('-')[2])
        if month == 9:
            num = day
        elif month == 10:
            num = day+30
        elif month == 11:
            num = day+61
        indexs.append(num-15)
    return indexs

nums = date2num(index)
device_2400_day['dateNum'] = nums

label = []
for d in device_2400_day['dateNum']:
    if d > 50: label.append(1)
    else: label.append(0)
device_2400_day['lable'] = label

In [4]:
device_7361 = data.loc[data['device_id'] == 7361]
device_7363 = data.loc[data['device_id'] == 7363]

device_7361 = device_7361.reset_index().drop('index', axis=1)
device_7361 = device_7361.drop('device_id', axis=1).drop('vc_id', axis=1).drop('vd_id', axis=1).drop('time', axis=1)

device_7363 = device_7363.reset_index().drop('index', axis=1)
device_7363 = device_7363.drop('device_id', axis=1).drop('vc_id', axis=1).drop('vd_id', axis=1).drop('time', axis=1)

device_7361_day = device_7361.groupby(['date']).sum()

device_7363_day = device_7361.groupby(['date']).sum()

device_7361_day['dateNum'] = nums

device_7363_day['dateNum'] = nums

device_7361_day['lable'] = label

device_7363_day['lable'] = label

In [5]:
train_data = device_2400_day.append(device_7361_day)
train_data.index = np.arange(train_data.shape[0])

test_data = device_7363_day
test_data.index = np.arange(test_data.shape[0])

train_data_X = train_data.drop(['lable'],axis=1)
train_data_Y = train_data['lable']
test_data_X = test_data.drop(['lable'],axis=1)
test_data_Y = test_data['lable']

In [6]:
need_drop_feature = ['net_droppedTx_summation',
                    'virtualDisk_numberReadAveraged_average', 
                    'virtualDisk_numberWriteAveraged_average', 
                    'virtualDisk_write_average',
                    'virtualDisk_read_average', 
                    'datastore_maxTotalLatency_latest', 
                    'net_droppedRx_summation', 
                    'dateNum']

for f in need_drop_feature:
    train_data_X = train_data_X.drop([f], axis=1)
    test_data_X = test_data_X.drop([f], axis=1)

In [7]:
# train_data_X.head()
print(train_data.shape)
print(test_data.shape)
print(train_data_X.shape)
print(test_data_X.shape)

(122, 16)
(61, 16)
(122, 7)
(61, 7)


In [8]:
# GBDT
from sklearn.ensemble import GradientBoostingClassifier

gbdt = GradientBoostingClassifier(learning_rate=0.7,max_depth=6,n_estimators=100,min_samples_leaf=2)

gbdt.fit(train_data_X,train_data_Y)

test_data["predict"] = gbdt.predict(test_data_X)
test_data.to_csv('gbdtTest2.csv')

In [9]:
# !/usr/bin/env python3
# coding=utf-8
"""
Decision Tree on the Basis of sklearn module
Author  :Chai Zheng
Blog    :http://blog.csdn.net/chai_zheng/
Github  :https://github.com/Chai-Zheng/Machine-Learning
Email   :zchaizju@gmail.com
Date    :2017.10.13
"""

import os
import time
import pydotplus
import numpy as np
from sklearn import tree
from sklearn.externals.six import StringIO
from sklearn.model_selection import train_test_split

print('Step 1.Loading data...')
X_train,X_test,Y_train,Y_test = train_data_X, test_data_X, train_data_Y, test_data_Y
print('---Loading and splitting completed.')

print('Step 2.Training...')
startTime = time.time()
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train,Y_train)
print('---Training Completed.Took %f s.'%(time.time()-startTime))

print('Step 3.Testing...')
Y_predict = clf.predict(X_test)
matchCount = 0
for i in range(len(Y_predict)):
    if Y_predict[i] == Y_test[i]:
        matchCount += 1
accuracy = float(matchCount/len(Y_predict))
print('---Testing completed.Accuracy: %.3f%%'%(accuracy*100))

feature_name = [ #"net_droppedTx_summation", 
                            "net_packetsTx_summation", 
#                             "virtualDisk_numberReadAveraged_average", 
#                             "virtualDisk_numberWriteAveraged_average",
#                             "virtualDisk_write_average",
#                             "virtualDisk_read_average",
#                             "datastore_maxTotalLatency_latest",
                            "cpu_usage_average",
                            "mem_usage_average",
                            "net_usage_average",
                            "net_packetsRx_summation",
#                             "net_droppedRx_summation",
                            "net_bytesRx_average",
                            "net_bytesTx_average"
                           ]
target_name = ['Class0', 'Class1']

dot_data = StringIO()
tree.export_graphviz(clf,out_file = dot_data,feature_names=feature_name,
                     class_names=target_name,filled=True,rounded=True,
                     special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
graph.write_pdf("WineTree2.pdf")
print('Visible tree plot saved as pdf.')

Step 1.Loading data...
---Loading and splitting completed.
Step 2.Training...
---Training Completed.Took 0.003004 s.
Step 3.Testing...
---Testing completed.Accuracy: 100.000%
Visible tree plot saved as pdf.


In [10]:
Y_predict = clf.predict(X_test)
Y_predict

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [35]:
sample0 = pd.DataFrame([[79397399.0, 195588.0, 304122.0, 9992298.0, 89428553.0, 873397.0, 2456465.0]])
sample1 = pd.DataFrame([[1465851.0, 20583.0, 301541.0, 595662.0, 1786858.0, 32061.0, 165232.0]])
predict0 = clf.predict(sample0)
predict1 = clf.predict(sample1)
print(predict0[0])
print(predict1[0])

0
1


In [40]:
sample1 = [79397399.0, 195588.0, 304122.0, 9992298.0, 89428553.0, 873397.0, 2456465.0]
sample2 = [1465851.0, 20583.0, 301541.0, 595662.0, 1786858.0, 32061.0, 165232.0]
samples = pd.DataFrame([sample1, sample2])
predicts = clf.predict(samples)
print(predicts)

[0 1]
